In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q simpletransformers scikit-learn pandas numpy tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 136.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 128.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.8 MB/s eta 0:00:00

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from simpletransformers.classification import ClassificationModel
import torch
import logging
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical


In [5]:
use_cuda = torch.cuda.is_available()
print("GPU Available:", use_cuda)


GPU Available: True


In [7]:
train_df = pd.read_csv("/content/drive/MyDrive/meme_classifications/ocr_text/train/STask_D_train.csv")
eval_df = pd.read_csv("/content/drive/MyDrive/meme_classifications/ocr_text/eval/STask-D(index,text)val.csv")
train_df.columns = ["index", "text", "labels"]
eval_df.columns = ["index", "text"]


In [8]:
label_encoder = LabelEncoder()
train_df["labels"] = label_encoder.fit_transform(train_df["labels"])
num_labels = len(label_encoder.classes_)


In [9]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["text"], train_df["labels"], test_size=0.1, random_state=42
)
train_data = pd.DataFrame({"text": train_texts, "labels": train_labels})
val_data = pd.DataFrame({"text": val_texts, "labels": val_labels})


In [10]:
albert_model = ClassificationModel(
    "albert",
    "albert-base-v2",
    num_labels=num_labels,
    args={
        "reprocess_input_data": True,
        "overwrite_output_dir": True,
        "num_train_epochs": 4,
        "learning_rate": 1e-5,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "max_seq_length": 128,
        "save_model_every_epoch": False,
        "save_eval_checkpoints": False,
        "use_multiprocessing": False,
        "fp16": False,
        "evaluate_during_training": True,
        "use_early_stopping": True,
        "early_stopping_patience": 2,
        "output_dir": "outputs_subtaskD",
    },
    use_cuda=use_cuda,
)

albert_model.train_model(train_data, eval_df=val_data)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Running Epoch 3 of 4:   0%|          | 0/228 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Running Epoch 4 of 4:   0%|          | 0/228 [00:00<?, ?it/s]

0it [00:00, ?it/s]

(912,
 defaultdict(list,
             {'global_step': [228, 456, 684, 912],
              'train_loss': [0.4337661564350128,
               0.3798360526561737,
               0.5334472060203552,
               0.26704326272010803],
              'mcc': [0.0,
               np.float64(0.12708928118947888),
               np.float64(0.23395341593155367),
               np.float64(0.26906429001404447)],
              'accuracy': [0.6592592592592592,
               0.6617283950617284,
               0.6864197530864198,
               0.6888888888888889],
              'f1_score': [0.39732142857142855,
               0.5164326613852067,
               0.5955127751433222,
               0.6287539285298568],
              'tp': [np.int64(267),
               np.int64(245),
               np.int64(235),
               np.int64(221)],
              'tn': [np.int64(0), np.int64(23), np.int64(43), np.int64(58)],
              'fp': [np.int64(138), np.int64(115), np.int64(95), np.int64(80)],
     

In [17]:
# Get ALBERT raw logits
albert_logits, _ = albert_model.predict(eval_df["text"].tolist())

from scipy.special import expit  # sigmoid

# Convert logits to probabilities for binary classification
probs_class1 = expit(albert_logits)  # sigmoid for class 1
probs_class0 = 1 - probs_class1      # remaining for class 0

# Combine into (N, 2) shape
albert_probs = np.vstack((probs_class0, probs_class1)).T



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [18]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(train_df["text"]).toarray()
y_train_keras = to_categorical(train_df["labels"])

# ANN Model
ann_model = Sequential()
ann_model.add(Dense(256, input_dim=5000, activation='relu'))
ann_model.add(Dense(num_labels, activation='softmax'))

ann_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
ann_model.fit(X_train_tfidf, y_train_keras, epochs=5, batch_size=32, verbose=1)


Epoch 1/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6766 - loss: 0.6343
Epoch 2/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7708 - loss: 0.4637
Epoch 3/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9002 - loss: 0.2800
Epoch 4/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9451 - loss: 0.1792
Epoch 5/5
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9682 - loss: 0.1156


In [19]:
X_eval_tfidf = tfidf.transform(eval_df["text"]).toarray()
keras_probs = ann_model.predict(X_eval_tfidf)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [20]:
# Average the probabilities


ensemble_probs = (albert_probs + keras_probs) / 2
ensemble_preds = np.argmax(ensemble_probs, axis=1)
ensemble_labels = label_encoder.inverse_transform(ensemble_preds)


In [21]:
submission_df = pd.DataFrame({
    "index": eval_df["index"],
    "label": ensemble_labels
})



submission_path = "/content/drive/MyDrive/meme_classifications/submissions/subtaskD_submission.csv"
submission_df.to_csv(submission_path, index=False)

print("✅ Submission saved to:", submission_path)


✅ Submission saved to: /content/drive/MyDrive/meme_classifications/submissions/subtaskD_submission.csv
